# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [1]:
# YOUR CHANGES HERE
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
import yfinance as yf

tickers = ["AAPL", "SPY", "TSLA", "RBLX", "R", "ROL"]

historical_prices = None

for t in tickers:
    ticker = yf.Ticker(t)
    history =  ticker.history(period="2y")

    monthly_prices = history['Close'].resample('ME').last()

    if historical_prices is None:
        historical_prices = monthly_prices
    else:
        historical_prices = pd.concat([historical_prices, monthly_prices], axis=1)

historical_prices.columns = tickers
historical_prices.index = pd.to_datetime(historical_prices.index).strftime("%Y-%m-%d")

historical_prices


,AAPL,SPY,TSLA,RBLX,R,ROL
Date,,,,,,
2023-09-30,169.549286,417.865692,250.220001,28.959999,102.248909,36.365944
2023-10-31,169.113525,408.794373,200.839996,31.809999,93.252541,36.638702
2023-11-30,188.355316,446.135162,240.080002,39.310001,103.127495,39.844788
2023-12-31,190.913666,466.503662,248.479996,45.720001,110.750900,42.710396
2024-01-31,182.851898,473.933441,187.289993,38.810001,109.316689,42.358311
2024-02-29,179.461273,498.666504,201.880005,39.900002,110.543343,43.259682
2024-03-31,170.257355,514.973938,175.789993,38.180000,116.443512,45.419235
2024-04-30,169.115555,494.210327,183.279999,35.560001,118.051758,43.740681
2024-05-31,191.138168,519.207336,178.080002,33.619999,118.355980,44.994606


Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [2]:
# YOUR CHANGES HERE

historical_prices.to_csv("historical_prices.tsv", sep="\t")

Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [3]:
# YOUR CHANGES HERE

historical_returns = historical_prices.pct_change().dropna()
historical_returns

,AAPL,SPY,TSLA,RBLX,R,ROL
Date,,,,,,
2023-10-31,-0.002570,-0.021709,-0.197346,0.098412,-0.087985,0.007500
2023-11-30,0.113780,0.091344,0.195379,0.235775,0.105895,0.087505
2023-12-31,0.013583,0.045655,0.034988,0.163063,0.073922,0.071919
2024-01-31,-0.042227,0.015927,-0.246257,-0.151137,-0.012950,-0.008244
2024-02-29,-0.018543,0.052187,0.077901,0.028086,0.011221,0.021280
2024-03-31,-0.051286,0.032702,-0.129235,-0.043108,0.053374,0.049921
2024-04-30,-0.006706,-0.040320,0.042608,-0.068622,0.013811,-0.036957
2024-05-31,0.130222,0.050580,-0.028372,-0.054556,0.002577,0.028667
2024-06-30,0.095553,0.035280,0.111186,0.106782,0.019840,0.067849


Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [4]:
# YOUR CHANGES HERE

historical_returns.to_csv("historical_returns.tsv", sep="\t")

Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [5]:
# YOUR CHANGES HERE

estimated_returns = historical_returns.mean()
estimated_returns = estimated_returns.to_frame("estimated_return")
estimated_returns = estimated_returns.rename_axis("asset")
print(estimated_returns)

       estimated_return
asset                  
AAPL           0.016170
SPY            0.019110
TSLA           0.024840
RBLX           0.072524
R              0.028064
ROL            0.019743


Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [6]:
# YOUR CHANGES HERE

estimated_returns.to_csv("estimated_returns.tsv", sep="\t")

Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [7]:
# YOUR CHANGES HERE

stock_covariances = historical_returns.cov()
stock_covariances

,AAPL,SPY,TSLA,RBLX,R,ROL
AAPL,0.003482,0.000786,0.003105,-0.000479,0.001463,0.000197
SPY,0.000786,0.001244,0.002322,0.001591,0.001674,0.000715
TSLA,0.003105,0.002322,0.025548,0.006712,0.004580,0.000259
RBLX,-0.000479,0.001591,0.006712,0.017920,0.002472,0.000049
R,0.001463,0.001674,0.004580,0.002472,0.004686,0.000805
ROL,0.000197,0.000715,0.000259,0.000049,0.000805,0.001980


Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [8]:
# YOUR CHANGES HERE

stock_covariances.to_csv("estimated_covariance.tsv", sep="\t")

Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [9]:
# YOUR CHANGES HERE

n = len(estimated_returns)

x = cp.Variable(n)

obj = cp.Maximize(estimated_returns.to_numpy().reshape(1, -1) @ x)

prob = cp.Problem(obj, [0 <= x, cp.sum(x) == 1])

estimated_return_P = prob.solve()
estimated_return_P

x.value.round(2)

array([0., 0., 0., 1., 0., 0.])

Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [10]:
# YOUR CHANGES HERE

max_return_ratio = pd.Series(np.asarray(x.value).ravel(), index=estimated_returns.index, name="allocation")

max_return_ratio.to_csv("maximum_return.tsv", sep="\t")

Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum return portfolio based on your previously estimated risks and returns.

In [11]:
# YOUR CHANGES HERE

x_min_risk = cp.Variable(n)

obj_min_risk = cp.Minimize(x_min_risk.T @ stock_covariances.to_numpy() @ x_min_risk)

prob_minimum_risk = cp.Problem(obj_min_risk,
                               [0 <= x_min_risk,
                                cp.sum(x_min_risk) == 1])

covariance_minimum_risk = prob_minimum_risk.solve()




Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [12]:
# YOUR CHANGES HERE

min_risk_ratio = pd.Series(np.asarray(x_min_risk.value).ravel(), index=estimated_returns.index, name="allocation")

min_risk_ratio.to_csv("minimum_risk.tsv", sep="\t")


Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [13]:
# YOUR CHANGES HERE

frontier_variances = []
frontier_returns = []
frontier_portfolios = []

estimated_return_minimum_risk = x_min_risk.value.T @ estimated_returns
estimated_return_minimum_risk = estimated_return_minimum_risk.astype(float)

historical_returns_ef = historical_prices.pct_change().dropna()
historical_returns_ef

,AAPL,SPY,TSLA,RBLX,R,ROL
Date,,,,,,
2023-10-31,-0.002570,-0.021709,-0.197346,0.098412,-0.087985,0.007500
2023-11-30,0.113780,0.091344,0.195379,0.235775,0.105895,0.087505
2023-12-31,0.013583,0.045655,0.034988,0.163063,0.073922,0.071919
2024-01-31,-0.042227,0.015927,-0.246257,-0.151137,-0.012950,-0.008244
2024-02-29,-0.018543,0.052187,0.077901,0.028086,0.011221,0.021280
2024-03-31,-0.051286,0.032702,-0.129235,-0.043108,0.053374,0.049921
2024-04-30,-0.006706,-0.040320,0.042608,-0.068622,0.013811,-0.036957
2024-05-31,0.130222,0.050580,-0.028372,-0.054556,0.002577,0.028667
2024-06-30,0.095553,0.035280,0.111186,0.106782,0.019840,0.067849


In [14]:

estimated_return_ef = historical_returns_ef.mean()
estimated_return_maximum_return = estimated_return_ef["RBLX"]


In [15]:

for r in np.linspace(estimated_return_minimum_risk, estimated_return_maximum_return, 101):

    x_r = cp.Variable(n)

    prob_r = cp.Problem(cp.Minimize(x_r.T @ stock_covariances.to_numpy() @ x_r),
                        [0 <= x_r,
                         cp.sum(x_r) == 1,
                         x_r.T @ estimated_returns == r])
    frontier_variances.append(prob_r.solve())
    frontier_returns.append(r)
    frontier_portfolios.append(x_r.value)

In [16]:
index = np.linspace(0,100,101)

df_ef = pd.DataFrame(list(zip(index, frontier_returns, frontier_variances)), columns=["index","return","risk"]).set_index("index")
df_ef["return"] = np.array(df_ef["return"].tolist(), dtype=float)
df_ef["risk"] = np.sqrt(df_ef["risk"])
df_ef

,return,risk
index,,
0.0,0.019572,0.031832
1.0,0.020102,0.031855
2.0,0.020631,0.031924
3.0,0.021161,0.032039
4.0,0.021690,0.032200
...,...,...
96.0,0.070406,0.128406
97.0,0.070936,0.129764
98.0,0.071465,0.131126


In [17]:
stacked = np.vstack([np.asarray(w).ravel() for w in frontier_portfolios])

allocations = pd.DataFrame(stacked, columns=list(estimated_returns.index))
allocations.index.name = "index"

allocations

,AAPL,SPY,TSLA,RBLX,R,ROL
index,,,,,,
0,1.579488e-01,5.204795e-01,-1.282765e-19,0.013701,-6.891517e-20,3.078708e-01
1,1.605898e-01,5.011545e-01,-1.160969e-19,0.023680,-6.254716e-20,3.145755e-01
2,1.632307e-01,4.818295e-01,-1.039186e-19,0.033660,-5.618049e-20,3.212802e-01
3,1.658717e-01,4.625045e-01,-9.174563e-20,0.043639,-4.981573e-20,3.279849e-01
4,1.685127e-01,4.431795e-01,-7.956531e-20,0.053618,-4.344699e-20,3.346896e-01
...,...,...,...,...,...,...
96,-7.965950e-24,-6.366236e-24,-2.069113e-24,0.952361,4.763948e-02,-8.073535e-24
97,5.712567e-25,-4.635789e-25,1.573027e-24,0.964270,3.572961e-02,-1.695855e-24
98,-6.311450e-24,-7.726563e-24,2.780540e-24,0.976180,2.381974e-02,-4.577729e-24


In [18]:
df_ef1 = pd.concat([df_ef, allocations], axis = 1)
df_ef1

,return,risk,AAPL,SPY,TSLA,RBLX,R,ROL
index,,,,,,,,
0.0,0.019572,0.031832,1.579488e-01,5.204795e-01,-1.282765e-19,0.013701,-6.891517e-20,3.078708e-01
1.0,0.020102,0.031855,1.605898e-01,5.011545e-01,-1.160969e-19,0.023680,-6.254716e-20,3.145755e-01
2.0,0.020631,0.031924,1.632307e-01,4.818295e-01,-1.039186e-19,0.033660,-5.618049e-20,3.212802e-01
3.0,0.021161,0.032039,1.658717e-01,4.625045e-01,-9.174563e-20,0.043639,-4.981573e-20,3.279849e-01
4.0,0.021690,0.032200,1.685127e-01,4.431795e-01,-7.956531e-20,0.053618,-4.344699e-20,3.346896e-01
...,...,...,...,...,...,...,...,...
96.0,0.070406,0.128406,-7.965950e-24,-6.366236e-24,-2.069113e-24,0.952361,4.763948e-02,-8.073535e-24
97.0,0.070936,0.129764,5.712567e-25,-4.635789e-25,1.573027e-24,0.964270,3.572961e-02,-1.695855e-24
98.0,0.071465,0.131126,-6.311450e-24,-7.726563e-24,2.780540e-24,0.976180,2.381974e-02,-4.577729e-24


Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [19]:
# YOUR CHANGES HERE

df_ef1.to_csv("efficient_frontier.tsv", sep="\t")

Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [43]:
# YOUR CHANGES HERE

rng = 42
T = 23
N = 1000 

np.random.seed(rng)

est_returns = estimated_returns.astype(float)
est_returns = estimated_returns.squeeze()
tickers = est_returns.index.astype(str)

covs = stock_covariances.reindex(index=tickers, columns=tickers)

covs = 0.5 * (covs + covs.T)
min_eig = np.linalg.eigvalsh(covs).min()
if min_eig < 0:
    covs -= (min_eig - 1e-12) * np.eye(len(est_returns))

wins = pd.Series(0, index=est_returns.index, dtype=int)

for _ in range(N):
    samples = np.random.multivariate_normal(
        mean=est_returns.to_numpy(),
        cov=covs,
        size=T)
    return_vector = samples.mean(axis=0)
    wins.iloc[np.argmax(return_vector)] += 1

ticker_stability = pd.DataFrame({
    "probability": wins / N
})

ticker_stability

,probability
asset,
AAPL,0.025
SPY,0.001
TSLA,0.078
RBLX,0.853
R,0.029
ROL,0.014


Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [44]:
# YOUR CHANGES HERE
ticker_stability.to_csv("max_return_probabilities.tsv", sep="\t")


Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgements.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.